In [2]:
from trace_generator import *

import random
import numpy
import pandas

In [3]:
tree = construct_random_tree(775451721, 548953530, tree_parameters={**default_tree_parameters, 'silent': 0.})

In [4]:
import pprint

def wrapper_time_func_with_rand(rand_val):
    return lambda x: (0.5 * x + numpy.random.uniform(low=0.01, high=0.1)) if rand_val >= x else ((x - rand_val)**2 + rand_val * 0.5 + numpy.random.uniform(low=0.01, high=0.1))


def wrapper_time_func(rand_val):
    return lambda x: (0.5 * x) if rand_val >= x else ((x - rand_val)**2 + rand_val * 0.5)


def wrapper_cost_func(rand_val):
    return lambda x: (-0.1 * numpy.sqrt(x)) if rand_val >= x else (-0.1 * numpy.sqrt(rand_val))

tt = 1718538115
tt2 = 1176869713

numpy.random.seed(tt)
random.seed(tt2)

tree_with_sim_properties = convert_pt_to_list_and_attach_properties(tree)

numpy.random.seed(tt)
random.seed(tt2)

activities_list = get_element_from_pt(tree_with_sim_properties, prefix='a_')

modelled_activities_list = generate_activity_simulation_values(
    activities_list,
    time_modifier_function=[lambda x: (0.5 / x) - 0.5, lambda x: numpy.power(x, 0.25) - 1],
    time_modifier_function_probability_distribution=[2/3, 1/3],
    # time_modifier_function_probability_distribution=None,
    cost_modifier_function=[lambda x: -0.05 * numpy.power(x, 1.25), lambda x: -0.15 * numpy.power(x, 1.5)],
    # cost_modifier_function=None,
    cost_modifier_function_probability_distribution=[4/5, 1/5]
)

time_modifier_rand_vals = dict()
for i in range(0, len(activities_list)):
    rand_val = numpy.random.randint(low=3, high=7)
    time_modifier_rand_vals[wrapper_time_func(rand_val)] = rand_val

cost_modifier_rand_vals = dict()
for i in range(0, len(activities_list)):
    rand_val = numpy.random.randint(low=5, high=10)
    cost_modifier_rand_vals[wrapper_cost_func(rand_val)] = rand_val

modelled_resources_list = generate_resources_simulation_values(
    activities_list,
    resources_limited_to_activities='s',
    apply_jack_of_all_trades_penalty=True,
    time_modifier_function=list(time_modifier_rand_vals.keys()),
    cost_modifier_function=list(cost_modifier_rand_vals.keys())
)

# Resources which are better at time efficiency but worse for cost.
modelled_resources_list += generate_resources_simulation_values(
    activities_list,
    start_resource_iter=len(modelled_resources_list),
    min_resources=2,
    max_resources=4,
    min_time_base_modifier=-0.5,
    max_time_base_modifier=-0.5,
    min_cost_base_modifier=0.5,
    max_cost_base_modifier=0.5,
    apply_jack_of_all_trades_penalty=False,
    resources_limited_to_activities='n',
    time_modifier_function=[lambda x: 0.1 * numpy.sqrt(x)],
    cost_modifier_function=[lambda x: -0.2 * numpy.sqrt(x)]
)

# Resources which are better at cost efficiency but worse for time.
modelled_resources_list += generate_resources_simulation_values(
    activities_list,
    start_resource_iter=len(modelled_resources_list),
    min_resources=2,
    max_resources=4,
    min_time_base_modifier=0.5,
    max_time_base_modifier=0.5,
    min_cost_base_modifier=-0.5,
    max_cost_base_modifier=-0.5,
    apply_jack_of_all_trades_penalty=False,
    resources_limited_to_activities='n',
    time_modifier_function=[lambda x: 0.1 * x**2],
    cost_modifier_function=[lambda x: -0.01 * numpy.sqrt(x)]
)

modelled_resources_list_dict = list()
for res in modelled_resources_list:
    modelled_resources_list_dict.append({
        'id': res[0],
        'name': res[1],
        'act_limit': res[2],
        'alloc_prob': res[3],
        'time_mod': res[4],
        'time_f': res[5],
        'cost_mod': res[6],
        'cost_f': res[7],
        'threshold_time': time_modifier_rand_vals[res[5]] if res[5] in time_modifier_rand_vals else None,
        'threshold_cost': cost_modifier_rand_vals[res[7]] if res[7] in cost_modifier_rand_vals else None
    })

pandas.DataFrame(data=modelled_resources_list_dict).drop(columns=['time_f', 'cost_f'])

,id,name,act_limit,alloc_prob,time_mod,cost_mod,threshold_time,threshold_cost
0,0,r_0,[a_a],0.158015,-0.010268,0.132464,3.0,7.0
1,1,r_1,[a_g],0.781009,0.216598,0.042697,4.0,7.0
2,2,r_2,"[a_e, a_c, a_f, a_g, a_d]",0.236172,-0.063923,0.086787,4.0,6.0
3,3,r_3,[a_f],0.832822,0.216360,0.107416,3.0,7.0
4,4,r_4,"[a_g, a_e, a_d]",0.794436,-0.292807,0.156056,3.0,6.0
5,5,r_5,"[a_d, a_c, a_e, a_a]",0.129981,0.297535,0.114275,4.0,8.0
6,6,r_6,[a_e],0.949316,0.167332,0.188116,4.0,8.0
7,7,r_7,"[a_d, a_a, a_f]",0.157495,-0.242720,0.077790,6.0,7.0
8,8,r_8,None,0.501517,0.012107,0.393627,4.0,8.0
9,9,r_9,None,0.802496,0.390332,0.345039,3.0,6.0
